In [10]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [133]:
# load datasets
aisles = pd.read_csv('datasets/aisles.csv')
departments = pd.read_csv('datasets/departments.csv')
orders = pd.read_csv('datasets/orders.csv')
products = pd.read_csv('datasets/products.csv')
order_products_train = pd.read_csv('datasets/order_products__train.csv')
order_products_prior = pd.read_csv('datasets/order_products__prior.csv')

In [134]:

def reduce_mem_usage(df: pd.DataFrame, name: str) -> pd.DataFrame:
  """
  ### Description
  Helper function that reduces the memory usage of large datasets. 

  ### Arguments:
  - `df`: A pandas dataframe
  - `name`: Name of the dataframe

  ### Returns:
  - `df`: A preprocessed pandas dataframe
  """
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              # np.iinfo is used to get the machine limits for integer types
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)  
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  print(f"{name}")
  print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.\
        format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  print("")
  return df

In [135]:
orders = reduce_mem_usage(orders, "Orders")
products = reduce_mem_usage(products, "Products")
order_products_train = reduce_mem_usage(order_products_train, "Order_Products")
order_products_prior = reduce_mem_usage(order_products_prior, "Order_Products_Prior")

Orders
Mem. usage decreased to 68.51 Mb (62.5% reduction)

Products
Mem. usage decreased to  0.71 Mb (53.1% reduction)

Order_Products
Mem. usage decreased to 13.20 Mb (68.7% reduction)

Order_Products_Prior
Mem. usage decreased to 340.25 Mb (65.6% reduction)



### **Approach**

*Prior* orders describe the past behaviour of a user while *train* and *test* orders regard the future behaviour that we need to predict. As a result, we want to predict which previously purchased products (*prior* orders) will be in a user’s next order (train and test orders). Since each user has purchased various products during their prior orders, this will be a classification problem because we need to predict whether each pair of user and product is a reorder or not (`reordered`). 

#### **Product-level features**

In [14]:
# for prior evaluation set
product_features_prior = order_products_prior.groupby(['product_id']) \
                            .agg(p_NumOrders=('product_id', 'count'), 
                                 p_NumReorders=('reordered', 'sum'),
                                 p_MedianAddToCartOrder=('add_to_cart_order', 'median')) \
                            .merge(products, how='left', on='product_id') \
                            .merge(aisles, how='left', on='aisle_id') \
                            .merge(departments, how='left', on='department_id')
product_features_prior['p_reorder_rate'] = product_features_prior['p_NumReorders'] / product_features_prior['p_NumOrders']
product_features_prior.sort_values('p_reorder_rate', ascending=False, inplace=True)

print(f"Shape of dataset: {product_features_prior.shape}")
print("")
print(f"Distribution for no. of orders: {product_features_prior['p_NumOrders'].describe()}")
print("")
product_features_prior.head(20)

Shape of dataset: (49677, 10)

Distribution for no. of orders: count     49677.000000
mean        652.907563
std        4792.114416
min           1.000000
25%          17.000000
50%          60.000000
75%         260.000000
max      472565.000000
Name: p_NumOrders, dtype: float64



,product_id,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,product_name,aisle_id,department_id,aisle,department,p_reorder_rate
6430,6433,68,64,2.0,Raw Veggie Wrappers,13,20,prepared meals,deli,0.941176
2074,2075,87,81,3.0,Serenity Ultimate Extrema Overnight Pads,126,11,feminine care,personal care,0.931034
43545,43553,13,12,2.0,Orange Energy Shots,64,7,energy sports drinks,beverages,0.923077
27734,27740,101,93,2.0,Chocolate Love Bar,45,19,candy chocolate,snacks,0.920792
14605,14609,35,32,1.0,Soy Powder Infant Formula,92,18,baby food formula,babies,0.914286
13871,13875,45,41,2.0,Simply Sleep Nighttime Sleep Aid,6,2,other,other,0.911111
39984,39992,22,20,3.0,"Energy Shot, Grape Flavor",64,7,energy sports drinks,beverages,0.909091
5865,5868,30,27,2.0,Russian River Valley Reserve Pinot Noir,28,5,red wines,alcohol,0.900000
31412,31418,60,54,3.0,Sparking Water,115,7,water seltzer sparkling water,beverages,0.900000
35598,35604,100,90,3.0,Maca Buttercups,45,19,candy chocolate,snacks,0.900000


As done in `1_EDA_and_Data_Cleaning.ipynb` reorder rates become pretty meaningless if they've only been ordered for only small amount of times. Hence, we'll aim to only look at `reorder_rate`'s for products with more orders than the 3rd quartile of `NumOrders` (~260).

In [15]:
product_features_prior.loc[product_features_prior['p_NumOrders'] < 260, 'p_reorder_rate'] = 0

In [16]:
# feature engineer the food vs non-food column
product_features_prior.department.unique()

array(['deli', 'personal care', 'beverages', 'snacks', 'babies', 'other',
       'alcohol', 'pantry', 'missing', 'pets', 'frozen', 'dairy eggs',
       'canned goods', 'produce', 'international', 'household',
       'dry goods pasta', 'breakfast', 'bakery', 'meat seafood', 'bulk'],
      dtype=object)

In [17]:
food_dept = [
    'deli', 'beverages', 'snacks', 'alcohol',
    'frozen', 'dairy eggs', 'canned goods', 'pantry',
    'dry goods pasta', 'breakfast', 'bakery', 
    'meat seafood', 'bulk', 'produce'
]

product_features_prior['p_FoodItem'] = product_features_prior['department'] \
    .apply(lambda row: True if row in food_dept else False)

In [18]:
product_features_prior.sort_values('product_id')[:10]

,product_id,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,product_name,aisle_id,department_id,aisle,department,p_reorder_rate,p_FoodItem
0,1,1852,1136,4.0,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,0.613391,True
1,2,90,12,8.0,All-Seasons Salt,104,13,spices seasonings,pantry,0.000000,True
2,3,277,203,4.0,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,0.732852,True
3,4,329,147,8.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen,0.446809,True
4,5,15,9,6.0,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry,0.000000,True
5,6,8,3,11.0,Dry Nose Oil,11,11,cold flu allergy,personal care,0.000000,False
6,7,30,12,5.5,Pure Coconut Water With Orange,98,7,juice nectars,beverages,0.000000,True
7,8,165,83,6.0,Cut Russet Potatoes Steam N' Mash,116,1,frozen produce,frozen,0.000000,True
8,9,156,82,6.0,Light Strawberry Blueberry Yogurt,120,16,yogurt,dairy eggs,0.000000,True
9,10,2572,1304,7.0,Sparkling Orange Juice & Prickly Pear Beverage,115,7,water seltzer sparkling water,beverages,0.506998,True


#### **User-level features**

In [108]:
order_product = orders.merge(order_products_prior, on='order_id', how='inner')

print(f"Dataframe shape: {order_product.shape}")
order_product.head(3)

Dataframe shape: (32434489, 10)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0


In [75]:
user_features_prior = order_product.groupby('user_id')\
                                   .agg(user_total_orders=('order_number', 'max'), 
                                        user_reordered_ratio=('reordered', 'mean'))\
                                   .reset_index()

user_features_prior.head(3)

,user_id,user_total_orders,user_reordered_ratio
0,1,10,0.694915
1,2,14,0.476923
2,3,12,0.625000


#### **User-x-Product-level features**

In [76]:
user_product_habits = order_product.groupby(['user_id', 'product_id'])\
                                   .agg(user_product_total_purchase=('product_id', 'count'))\
                                   .reset_index()\
                                   .merge(user_features_prior, how='inner', on='user_id')\

user_product_habits.head(3)

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio
0,1,196,10,10,0.694915
1,1,10258,9,10,0.694915
2,1,10326,1,10,0.694915


In [77]:
user_product_habits['user_product_reorder_rate'] = user_product_habits['user_product_total_purchase'] / \
                                                user_product_habits['user_total_orders']

user_product_habits.head(3)

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate
0,1,196,10,10,0.694915,1.0
1,1,10258,9,10,0.694915,0.9
2,1,10326,1,10,0.694915,0.1


### **Consolidate**

In [78]:
all_features = user_product_habits.merge(product_features_prior, how='left', on='product_id')

print(f"Features dataframe shape{all_features.shape}")
all_features.head(3)

Features dataframe shape(13307953, 16)


,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,product_name,aisle_id,department_id,aisle,department,p_reorder_rate,p_FoodItem
0,1,196,10,10,0.694915,1.0,35791,27791,2.0,Soda,77,7,soft drinks,beverages,0.776480,True
1,1,10258,9,10,0.694915,0.9,1946,1389,3.0,Pistachios,117,19,nuts seeds dried fruit,snacks,0.713772,True
2,1,10326,1,10,0.694915,0.1,5526,3603,3.0,Organic Fuji Apples,24,4,fresh fruits,produce,0.652009,True


In [79]:
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]\
                                           [['user_id', 'eval_set', 'order_id']]

orders_future.head(3)

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568


In [109]:
# bring the info of the future orders to data DF
features_with_orders = all_features.merge(orders_future, on='user_id', how='left')
features_with_orders.drop(['department_id', 'product_name', 'aisle_id',
                           'aisle', 'department', ], axis=1, inplace=True)
features_with_orders.head(3)

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,p_reorder_rate,p_FoodItem,eval_set,order_id
0,1,196,10,10,0.694915,1.0,35791,27791,2.0,0.776480,True,train,1187899
1,1,10258,9,10,0.694915,0.9,1946,1389,3.0,0.713772,True,train,1187899
2,1,10326,1,10,0.694915,0.1,5526,3603,3.0,0.652009,True,train,1187899


In [130]:
order_product.query('user_id == 1 and product_id == 196')

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
11,473747,1,prior,3,3,12,21.0,196,1,1
16,2254736,1,prior,4,4,7,29.0,196,1,1
21,431534,1,prior,5,4,15,28.0,196,1,1
29,3367565,1,prior,6,2,7,19.0,196,1,1
33,550135,1,prior,7,1,9,20.0,196,1,1
39,3108588,1,prior,8,1,14,14.0,196,2,1
47,2295261,1,prior,9,1,16,0.0,196,4,1
50,2550362,1,prior,10,4,8,30.0,196,1,1


In [131]:
features_with_orders2 = features_with_orders.merge(order_product.groupby(['user_id', 'product_id']).agg(
                                                user_product_days_since_order=('days_since_prior_order', 'last')
                                            ),
                                            how="left",
                                            on=['user_id', 'product_id'])

features_with_orders2['user_product_days_since_order'].fillna(0, inplace=True)

features_with_orders2[:20]

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,p_reorder_rate,p_FoodItem,eval_set,order_id,user_product_days_since_order
0,1,196,10,10,0.694915,1.000000,35791,27791,2.0,0.776480,True,train,1187899,30.0
1,1,10258,9,10,0.694915,0.900000,1946,1389,3.0,0.713772,True,train,1187899,30.0
2,1,10326,1,10,0.694915,0.100000,5526,3603,3.0,0.652009,True,train,1187899,28.0
3,1,12427,10,10,0.694915,1.000000,6476,4797,3.0,0.740735,True,train,1187899,30.0
4,1,13032,3,10,0.694915,0.300000,3751,2465,4.0,0.657158,True,train,1187899,30.0
5,1,13176,2,10,0.694915,0.200000,379450,315913,3.0,0.832555,True,train,1187899,28.0
6,1,14084,1,10,0.694915,0.100000,15935,12923,4.0,0.810982,True,train,1187899,0.0
7,1,17122,1,10,0.694915,0.100000,13880,9377,4.0,0.675576,True,train,1187899,28.0
8,1,25133,8,10,0.694915,0.800000,6196,4586,5.0,0.740155,True,train,1187899,30.0
9,1,26088,2,10,0.694915,0.200000,2523,1360,5.0,0.539041,True,train,1187899,15.0


We'll now keep only the customers who we know what they bought in their future order (where `eval_set` = 0). Then, we'll join them with the `order_products_train` dataframe where we know whether they reordered the product or not.

In [140]:
data_train = features_with_orders2[features_with_orders2.eval_set=='train']\
                                .merge(order_products_train[['product_id','order_id', 'reordered']], 
                                       on=['product_id','order_id'], how='left' )
                                
data_train.head(10)

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,p_reorder_rate,p_FoodItem,eval_set,order_id,user_product_days_since_order,reordered
0,1,196,10,10,0.694915,1.0,35791,27791,2.0,0.776480,True,train,1187899,30.0,1.0
1,1,10258,9,10,0.694915,0.9,1946,1389,3.0,0.713772,True,train,1187899,30.0,1.0
2,1,10326,1,10,0.694915,0.1,5526,3603,3.0,0.652009,True,train,1187899,28.0,NaN
3,1,12427,10,10,0.694915,1.0,6476,4797,3.0,0.740735,True,train,1187899,30.0,NaN
4,1,13032,3,10,0.694915,0.3,3751,2465,4.0,0.657158,True,train,1187899,30.0,1.0
5,1,13176,2,10,0.694915,0.2,379450,315913,3.0,0.832555,True,train,1187899,28.0,NaN
6,1,14084,1,10,0.694915,0.1,15935,12923,4.0,0.810982,True,train,1187899,0.0,NaN
7,1,17122,1,10,0.694915,0.1,13880,9377,4.0,0.675576,True,train,1187899,28.0,NaN
8,1,25133,8,10,0.694915,0.8,6196,4586,5.0,0.740155,True,train,1187899,30.0,1.0
9,1,26088,2,10,0.694915,0.2,2523,1360,5.0,0.539041,True,train,1187899,15.0,1.0


We notice some null values in the `reordered` column. We can safely assume this means that the customers didn't end up reordering the product (leading it to not be present in the *train* set). Thus, we impute the column with 0s.

In [141]:
data_train['reordered'].fillna(0, inplace=True)
data_train.head(10)

,user_id,product_id,user_product_total_purchase,user_total_orders,user_reordered_ratio,user_product_reorder_rate,p_NumOrders,p_NumReorders,p_MedianAddToCartOrder,p_reorder_rate,p_FoodItem,eval_set,order_id,user_product_days_since_order,reordered
0,1,196,10,10,0.694915,1.0,35791,27791,2.0,0.776480,True,train,1187899,30.0,1.0
1,1,10258,9,10,0.694915,0.9,1946,1389,3.0,0.713772,True,train,1187899,30.0,1.0
2,1,10326,1,10,0.694915,0.1,5526,3603,3.0,0.652009,True,train,1187899,28.0,0.0
3,1,12427,10,10,0.694915,1.0,6476,4797,3.0,0.740735,True,train,1187899,30.0,0.0
4,1,13032,3,10,0.694915,0.3,3751,2465,4.0,0.657158,True,train,1187899,30.0,1.0
5,1,13176,2,10,0.694915,0.2,379450,315913,3.0,0.832555,True,train,1187899,28.0,0.0
6,1,14084,1,10,0.694915,0.1,15935,12923,4.0,0.810982,True,train,1187899,0.0,0.0
7,1,17122,1,10,0.694915,0.1,13880,9377,4.0,0.675576,True,train,1187899,28.0,0.0
8,1,25133,8,10,0.694915,0.8,6196,4586,5.0,0.740155,True,train,1187899,30.0,1.0
9,1,26088,2,10,0.694915,0.2,2523,1360,5.0,0.539041,True,train,1187899,15.0,1.0


We'll also set user_id and product_id as the index of the DF (for both *train* & *test*).

In [142]:
data_train = data_train.set_index(['user_id', 'product_id'])
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)

In [145]:
data_train.head(3)

user_product_total_purchase  user_total_orders  \
user_id product_id                                                   
1       196                                  10                 10   
        10258                                 9                 10   
        10326                                 1                 10   

                    user_reordered_ratio  user_product_reorder_rate  \
user_id product_id                                                    
1       196                     0.694915                        1.0   
        10258                   0.694915                        0.9   
        10326                   0.694915                        0.1   

                    p_NumOrders  p_NumReorders  p_MedianAddToCartOrder  \
user_id product_id                                                       
1       196               35791          27791                     2.0   
        10258              1946           1389                     3.0   
        10326              5526           3603                     3.0   

                    p_reorder_rate  p_FoodItem  user_product_days_since_order  \
user_id product_id                                                              
1       196               0.776480        True                           30.0   
        10258             0.713772        True                           30.0   
        10326             0.652009        True                           28.0   

                    reordered  
user_id product_id             
1       196               1.0  
        10258             1.0  
        10326             0.0

In [143]:
data_test = features_with_orders2[features_with_orders2.eval_set=='test']
data_test = data_test.set_index(['user_id', 'product_id'])
data_test = data_test.drop(['eval_set','order_id'], axis=1)

In [146]:
data_test.head(3)

user_product_total_purchase  user_total_orders  \
user_id product_id                                                   
3       248                                   1                 12   
        1005                                  1                 12   
        1819                                  3                 12   

                    user_reordered_ratio  user_product_reorder_rate  \
user_id product_id                                                    
3       248                        0.625                   0.083333   
        1005                       0.625                   0.083333   
        1819                       0.625                   0.250000   

                    p_NumOrders  p_NumReorders  p_MedianAddToCartOrder  \
user_id product_id                                                       
3       248                6371           2550                     9.0   
        1005                463            204                     8.0   
        1819               2424           1193                     7.0   

                    p_reorder_rate  p_FoodItem  user_product_days_since_order  
user_id product_id                                                             
3       248               0.400251        True                            9.0  
        1005              0.440605        True                           17.0  
        1819              0.492162        True                            7.0

### **Export**

In [147]:
data_test.to_parquet("datasets/test_set_features.parquet", index=True)
data_train.to_parquet("datasets/train_set_features.parquet", index=True)